In [32]:
#import ibapi
import ib_insync
from ib_insync import *
import pandas as pd
import concurrent

In [34]:
util.startLoop()  # blackmagic

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=1,readonly=True)

<IB connected to 127.0.0.1:7496 clientId=1>

In [55]:
#contract = Forex('EURUSD')
#contract=Stock('WINE', 'SMART', 'GBP')
contract=Stock('VZ', 'SMART', 'USD')

bars = ib.reqHistoricalData(
    contract, endDateTime='', durationStr='1 D',
    barSizeSetting='1 min', whatToShow='MIDPOINT', useRTH=True)

# convert to pandas dataframe:
df_mid = util.df(bars).set_index("date").rename(axis=1,mapper=lambda n: "mid "+n)

bars = ib.reqHistoricalData(
    contract, endDateTime='', durationStr='1 D',
    barSizeSetting='1 min', whatToShow='ASK', useRTH=True)

# convert to pandas dataframe:
df_ask = util.df(bars).set_index("date").rename(axis=1,mapper=lambda n: "ask "+n)

bars = ib.reqHistoricalData(
    contract, endDateTime='', durationStr='1 D',
    barSizeSetting='1 min', whatToShow='BID', useRTH=True)

df_bid = util.df(bars).set_index("date").rename(axis=1,mapper=lambda n: "bid "+n)

pd.concat([df_mid,df_ask,df_bid],axis=1)

Error 162, reqId 28: Historical Market Data Service error message:No market data permissions for NYSE STK. Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details., contract: Stock(symbol='VZ', exchange='SMART', currency='USD')


AttributeError: 'NoneType' object has no attribute 'set_index'

In [54]:
ticker=ib.reqMktDepth(contract)
util.df(ticker.domBids).drop(columns=["marketMaker"]).sort_values("price")
util.df(ticker.domAsks).drop(columns=["marketMaker"]).sort_values("price")

AttributeError: 'NoneType' object has no attribute 'drop'

Error 10092, reqId 27: Deep market data is not supported for this combination of security type/exchange, contract: Stock(symbol='VZ', exchange='SMART', currency='USD')


In [56]:
ticks=ib.reqHistoricalTicks(contract, '', datetime.datetime.now()-datetime.timedelta(hours=3), 1000, 'TRADES', False)
df=util.df(ticks)
df

Error 10187, reqId 29: Failed to request historical ticks:No market data permissions for NYSE STK, contract: Stock(symbol='VZ', exchange='SMART', currency='USD')


In [57]:
#ib.reqHistogramData(contract,useRTH=True,period="3 days")
a=ib.reqTickByTickData(Stock("T","SMART",currency="USD"),"AllLast")
a

Ticker(contract=Stock(symbol='T', exchange='SMART', currency='USD'))

Error 10189, reqId 30: Failed to request tick-by-tick data.No market data permissions for NYSE STK. Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details., contract: Stock(symbol='T', exchange='SMART', currency='USD')
Error 317, reqId 8: Market depth data has been RESET. Please empty deep book contents before applying any new entries., contract: Forex('EURUSD', exchange='IDEALPRO')
Error 317, reqId 9: Market depth data has been RESET. Please empty deep book contents before applying any new entries., contract: Forex('EURUSD', exchange='IDEALPRO')
Error 317, reqId 7: Market depth data has been RESET. Please empty deep book contents before applying any new entries., contract: Forex('EURUSD', exchange='IDEALPRO')
Error 1100, reqId -1: Connectivity between IB and TWS has been lost.


In [12]:
#returns html for god sake
newsProviders = ib.reqNewsProviders()
print(newsProviders)
codes = '+'.join(np.code for np in newsProviders)

stock = Stock('T', 'SMART', 'USD')
ib.qualifyContracts(stock)
headlines = ib.reqHistoricalNews(stock.conId, codes, '2020-10-10', '', 10)
latest = headlines[0]
print(latest)
article = ib.reqNewsArticle(latest.providerCode, latest.articleId)
print(article)

[NewsProvider(code='BRFG', name='Briefing.com General Market Columns'), NewsProvider(code='BRFUPDN', name='Briefing.com Analyst Actions'), NewsProvider(code='DJNL', name='Dow Jones Newsletters')]
HistoricalNews(time=datetime.datetime(2022, 7, 22, 11, 34, 40), providerCode='BRFUPDN', articleId='BRFUPDN$121e1888', headline='{A:800015:L:en:K:-0.97:C:0.97}!Barclays downgraded AT&amp;T (T) to Equal Weight with target $20')
NewsArticle(articleType=0, articleText='Barclays downgraded AT&amp;T (T) to Equal Weight rating with price target $20&#10;Previous rating: Overweight&#10;Previous price target: $22&#10;Issuance Date: 2022-07-22&#10;&#10;Copyright 2022 Briefing.com, Inc.')


In [25]:
def thread_distribute(l,f,n=1):
    #print(type(l))
    if(n==0):
        return f(l)
    
    with concurrent.futures.ThreadPoolExecutor() as executor: 
        l=[executor.submit(lambda x: thread_distribute(x,f,n-1),x) for x in l]
        l= [x.result() for x in l]
    return l

thread_distribute([range(5) for i in range(2)],lambda x: range(x),2)

[[range(0, 0), range(0, 1), range(0, 2), range(0, 3), range(0, 4)],
 [range(0, 0), range(0, 1), range(0, 2), range(0, 3), range(0, 4)]]

In [52]:
contr

Stock(symbol='VZ', exchange='SMART', currency='USD')